In [1]:
%pip install transformers
%pip install torchaudio
%pip install sounddevice scipy
%pip install soundfile
%pip install pyserial


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from transformers import (Wav2Vec2Processor, Wav2Vec2ForCTC)  # Import necessary classes from the transformers library
import torchaudio  # Import torchaudio for audio processing
import torch  # Import PyTorch for tensor operations

# Function to convert speech file to array
def speech_file_to_array_fn(voice_path, resampling_to=16000):
    # Load the audio file and get the waveform and sampling rate
    speech_array, sampling_rate = torchaudio.load(voice_path)
    
    # Initialize a resampling transform to resample audio if needed
    resampler = torchaudio.transforms.Resample(sampling_rate, resampling_to)

    # Resample the audio and convert to a NumPy array
    return resampler(speech_array)[0].numpy(), sampling_rate

# Define the pre-trained model checkpoint or identifier
cp = "bakrianoo/sinai-voice-ar-stt"

# Initialize Wav2Vec2 processor and model using the pre-trained checkpoint
processor = Wav2Vec2Processor.from_pretrained(cp)
model = Wav2Vec2ForCTC.from_pretrained(cp)
from transformers import (Wav2Vec2Processor, Wav2Vec2ForCTC)  # Import necessary classes from the transformers library
import torchaudio  # Import torchaudio for audio processing
import torch  # Import PyTorch for tensor operations

# Function to convert speech file to array
def speech_file_to_array_fn(voice_path, resampling_to=16000):
    # Load the audio file and get the waveform and sampling rate
    speech_array, sampling_rate = torchaudio.load(voice_path)
    
    # Initialize a resampling transform to resample audio if needed
    resampler = torchaudio.transforms.Resample(sampling_rate, resampling_to)

    # Resample the audio and convert to a NumPy array
    return resampler(speech_array)[0].numpy(), sampling_rate

# Define the pre-trained model checkpoint or identifier
cp = "bakrianoo/sinai-voice-ar-stt"

# Initialize Wav2Vec2 processor and model using the pre-trained checkpoint
processor = Wav2Vec2Processor.from_pretrained(cp)
model = Wav2Vec2ForCTC.from_pretrained(cp)


C:\Users\abdal\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at bakrianoo/sinai-voice-ar-stt were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a mo

Recording

In [4]:
import time
import sounddevice as sd
from scipy.io.wavfile import write

def record_audio(duration=5, filename="output.wav", sample_rate=44100):
    start_time = time.time()  # Start timing
    print("Recording...")
    recording = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=2)
    sd.wait()
    print("Recording finished")
    write(filename, sample_rate, recording)
    end_time = time.time()  # End timing
    execution_time = end_time - start_time
    print(f"Time taken for audio recording: {execution_time} seconds")


# Change the duration, filename, and sample rate as needed
record_duration = 3  # in seconds
output_filename = "my_recording.wav"
sample_rate = 44100  # 44100 is a common sample rate

record_audio(duration=record_duration, filename=output_filename, sample_rate=sample_rate)


Recording...
Recording finished
Time taken for audio recording: 3.340597629547119 seconds


Convert record to text

In [9]:
import torchaudio

# Assuming you have the necessary functions and models initialized beforehand
sound_path = 'my_recording.wav'

sample, sr = speech_file_to_array_fn(sound_path)
inputs = processor([sample], sampling_rate=16_000, return_tensors="pt", padding=True)
start_time = time.time()
with torch.no_grad():
    logits = model(inputs.input_values).logits

predicted_ids = torch.argmax(logits, dim=-1)

transcription_string = processor.batch_decode(predicted_ids)[0]  # Get the transcription as a string

print("Prediction:", transcription_string)
end_time = time.time()
execution_time = end_time - start_time
print(f"Time taken for speech-to-text conversion: {execution_time} seconds")

# Now 'transcription_string' contains the transcription as a string


Prediction: كيف حالك
Time taken for speech-to-text conversion: 0.9102780818939209 seconds


Send command to Arduino

In [63]:

import serial
import time

arduino_port = 'COM9'
baud_rate = 9600

ser = serial.Serial(arduino_port, baud_rate)
time.sleep(2)

command = transcription_string
ser.write(command.encode('utf-8') + b'\n')
print(f"Sent '{command}' command to Arduino")

# Close the serial connection
ser.close()


COM9 - USB-SERIAL CH340 (COM9)
COM9
Time taken for serial port communication: 0.0 seconds
Time taken for serial port communication: 0.0 seconds
Time taken for serial port communication: 0.002672433853149414 seconds
Time taken for serial port communication: 0.0019998550415039062 seconds
Time taken for serial port communication: 0.0019998550415039062 seconds
Time taken for serial port communication: 0.002000093460083008 seconds
Time taken for serial port communication: 0.0019998550415039062 seconds
Time taken for serial port communication: 0.0025115013122558594 seconds
Time taken for serial port communication: 0.0020112991333007812 seconds
Time taken for serial port communication: 0.0023081302642822266 seconds
Time taken for serial port communication: 0.0020341873168945312 seconds
Time taken for serial port communication: 0.0020008087158203125 seconds
Time taken for serial port communication: 0.0020275115966796875 seconds
Time taken for serial port communication: 0.0019998550415039062 se

KeyboardInterrupt: 

In [30]:
import time
import sounddevice as sd
import torchaudio
import serial
from scipy.io.wavfile import write

def record_audio(duration=5, filename="output.wav", sample_rate=44100):
    start_time = time.time()  # Start timing
    print("Recording...")
    recording = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=2)
    sd.wait()
    print("Recording finished")
    write(filename, sample_rate, recording)
    end_time = time.time()  # End timing
    execution_time = end_time - start_time
    print(f"Time taken for audio recording: {execution_time} seconds")


# Change the duration, filename, and sample rate as needed
record_duration = 3  # in seconds
output_filename = "my_recording.wav"
sample_rate = 44100  # 44100 is a common sample rate

record_audio(duration=record_duration, filename=output_filename, sample_rate=sample_rate)


# Assuming you have the necessary functions and models initialized beforehand
sound_path = 'my_recording.wav'

sample, sr = speech_file_to_array_fn(sound_path)
inputs = processor([sample], sampling_rate=16_000, return_tensors="pt", padding=True)
start_time = time.time()
with torch.no_grad():
    logits = model(inputs.input_values).logits

predicted_ids = torch.argmax(logits, dim=-1)

transcription_string = processor.batch_decode(predicted_ids)[0]  # Get the transcription as a string

print("Prediction:", transcription_string)
end_time = time.time()
execution_time = end_time - start_time
print(f"Time taken for speech-to-text conversion: {execution_time} seconds")

# Now 'transcription_string' contains the transcription as a string

arduino_port = 'COM8'
baud_rate = 9600

ser = serial.Serial(arduino_port, baud_rate)
time.sleep(2)

command = transcription_string
ser.write(command.encode('utf-8') + b'\n')
print(f"Sent '{command}' command to Arduino")

# Close the serial connection
ser.close()

Recording...
Recording finished
Time taken for audio recording: 3.327630043029785 seconds
Prediction: شركة
Time taken for speech-to-text conversion: 1.7378268241882324 seconds
Sent 'شركة' command to Arduino
